In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import math

from sklearn.datasets import fetch_20newsgroups
import numpy as np

import argparse
from types import SimpleNamespace

import random
random.seed(1234)

import pandas as pd
import matplotlib.pyplot as plt

from nltk.tokenize import RegexpTokenizer

import bsg, tools

import gensim.downloader as api
from gensim.models import Word2Vec, FastText, KeyedVectors

%load_ext line_profiler

In [3]:
princeton_data = pd.read_csv("articles_1999.csv", delimiter="\|\~\|", header=None)
princeton_data.columns = ["ID", "headline", "date", "type", "author", "author_title", "article", "author_bio"]
princeton_data = princeton_data[princeton_data["article"].notnull()]


/anaconda3/envs/math689env/lib/python3.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [32]:
prince_data = list(princeton_data.loc[0:40,"article"])

In [33]:
len(prince_data)

32

In [34]:

### Stuff ###

window = 2
t = np.power(10.0,-4.0)



In [35]:
vocabulary, tokenized_corpus, word2idx, idx2word = tools.tokenize_corpus(prince_data)

indexed_corpus = tools.index_tokenized_corpus(tokenized_corpus, word2idx)

flattened_indexed_corpus = tools.flatten(indexed_corpus)
unigram_subsampling = tools.make_unigram_dict(flattened_indexed_corpus)

subsampled_indexed_corpus = tools.subsample_corpus(indexed_corpus, unigram_subsampling, t)

sum([len(doc) for doc in subsampled_indexed_corpus])

len(flattened_indexed_corpus)

corpus_center_words, corpus_context_words = tools.get_corpus_centers_contexts(subsampled_indexed_corpus, window)

unigram = tools.make_unigram_dict(corpus_center_words)

In [36]:
corpus_center_words.shape

torch.Size([12784])

In [17]:
## w2v ##

In [18]:
# all_prince = list(princeton_data.loc[0:princeton_data.shape[0],6])
# vocabulary, tokenized_corpus, word2idx, idx2word = tools.tokenize_corpus(princeton_data["article"])



In [37]:
### word2vec ###

w2v = Word2Vec(sample = t, sg=1, negative=1, size=30, window=2, min_count=1, max_vocab_size=None, seed=1, workers=1)

w2v.build_vocab(tokenized_corpus)  # prepare the model vocabulary
w2v.train(tokenized_corpus, total_examples=w2v.corpus_count, epochs=50)


(540690, 954250)

In [ ]:
"barbara" in w2v.wv.vocab

In [69]:
w2v.wv.most_similar("jewish") # assault, financial, mental, income

[('stereotype', 0.6241565942764282),
 ('premier', 0.6206339597702026),
 ('papers', 0.6167088150978088),
 ('date', 0.5841314792633057),
 ('humans', 0.5464102029800415),
 ('town', 0.5446085333824158),
 ('vulnerability', 0.5382357835769653),
 ('cartoons', 0.5376099348068237),
 ('extension', 0.5208137035369873),
 ('eligible', 0.5180891752243042)]

In [ ]:
# # Get corpus: 20 news groups

# newsgroups_train = fetch_20newsgroups(subset='train')

# window = 5

# vocabulary, tokenized_corpus, word2idx, idx2word = tools.tokenize_corpus(newsgroups_train.data)

# indexed_corpus = tools.index_tokenized_corpus(tokenized_corpus, word2idx)

# corpus_center_words, corpus_context_words = tools.get_corpus_centers_contexts(indexed_corpus, window)

# unigram = tools.make_unigram_dict(corpus_center_words)

In [40]:
args_dict = {"vocab_size" : len(vocabulary), "window" : window, 
             "batch_size" : 200, "optimizer" : 80, "learning_rate" : 0.0005, 
             "momentum" : 0.99, "num_epoch" : 50, "init_mult" : 1, 
             "variance" : 0.995, "start" : True, "nogpu" : True, 
             "embedding_dim" : 300, "freeze" : False}

args = SimpleNamespace(**args_dict)

In [41]:


model = bsg.BSG(args.window, unigram, args.vocab_size, input_dim = 50, latent_dim = 30)
optimizer = torch.optim.Adam(model.parameters(), args.learning_rate, betas=(args.momentum, 0.999))
model = bsg.train(model, args, optimizer, corpus_center_words, corpus_context_words)



Epoch 0, loss=50344.76953125
Epoch 5, loss=26069.400390625
Epoch 10, loss=16207.2314453125
Epoch 15, loss=10887.6474609375
Epoch 20, loss=7951.52392578125
Epoch 25, loss=5915.08544921875
Epoch 30, loss=4590.451171875
Epoch 35, loss=3569.544677734375
Epoch 40, loss=2941.241455078125
Epoch 45, loss=2436.4189453125


In [27]:
# %lprun -T lprofencoder -f bsg.BSG.encoder model.encoder(corpus_center_words[0:2], corpus_context_words[0:2])



*** Profile printout saved to text file 'lprofencoder'. 


In [ ]:
model.encoder(corpus_center_words[0:2], corpus_context_words[0:2])

In [42]:
# torch.save(model, "bsg_prince_latent30")

In [ ]:
model = torch.load("bsg_prince_latent30")

In [49]:
# idx2word.get(corpus_center_words[0])

In [54]:
types, counts = np.unique(corpus_center_words.tolist(), return_counts=True)

In [55]:
test_idx2realidx = {idx: realidx for (idx , realidx) in enumerate(types)}
test_realidx2idx = {realidx: idx for (idx , realidx) in enumerate(types)}

In [101]:
test_idx2realidx

{0: 2,
 1: 4,
 2: 5,
 3: 6,
 4: 10,
 5: 16,
 6: 20,
 7: 21,
 8: 30,
 9: 33,
 10: 36,
 11: 42,
 12: 43,
 13: 46,
 14: 47,
 15: 48,
 16: 52,
 17: 53,
 18: 54,
 19: 65,
 20: 68,
 21: 73,
 22: 75,
 23: 76,
 24: 79,
 25: 88,
 26: 90,
 27: 92,
 28: 93,
 29: 105,
 30: 109,
 31: 113,
 32: 118,
 33: 119,
 34: 126,
 35: 127,
 36: 130,
 37: 137,
 38: 150,
 39: 166,
 40: 167,
 41: 168,
 42: 172,
 43: 173,
 44: 178,
 45: 180,
 46: 181,
 47: 187,
 48: 189,
 49: 190,
 50: 191,
 51: 194,
 52: 196,
 53: 203,
 54: 205,
 55: 206,
 56: 207,
 57: 220,
 58: 229,
 59: 233,
 60: 235,
 61: 236,
 62: 238,
 63: 239,
 64: 240,
 65: 241,
 66: 242,
 67: 245,
 68: 246,
 69: 254,
 70: 255,
 71: 257,
 72: 260,
 73: 261,
 74: 266,
 75: 268,
 76: 269,
 77: 270,
 78: 274,
 79: 275,
 80: 280,
 81: 281,
 82: 288,
 83: 293,
 84: 296,
 85: 310,
 86: 319,
 87: 322,
 88: 324,
 89: 326,
 90: 328,
 91: 334,
 92: 335,
 93: 336,
 94: 339,
 95: 343,
 96: 349,
 97: 352,
 98: 354,
 99: 355,
 100: 357,
 101: 358,
 102: 361,
 103: 363,

In [59]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim_matrix = cosine_similarity(model.type_means(torch.tensor(types)).detach().numpy(), 
                                   model.type_means(torch.tensor(types)).detach().numpy())
def n_closest_words(word, cos_sim_matrix, n):
    word_index = test_realidx2idx.get(word2idx.get(word))
    print("word_index is " + str(word_index))
    close_words_indices = np.argsort(cos_sim_matrix[word_index])[-n:]
#     print("close_words_indices.shape" + str(close_words_indices.shape))
    return [idx2word.get(test_idx2realidx.get(j)) for j in close_words_indices]

In [162]:
n_closest_words("pressure", cos_sim_matrix, 20) # assault, financial, mental, income

word_index is 917
close_words_indices.shape(20,)


['disappointment',
 'alongside',
 'opportunity',
 'safety',
 None,
 'significantly',
 'right',
 'content',
 None,
 'ivy',
 'norwegian',
 None,
 'overwhelmingly',
 'checked',
 'junior',
 'accept',
 'moral',
 'mice',
 'its',
 'pressure']

In [95]:
model.type_logvars(torch.tensor(word2idx.get("thing"))).detach().exp().numpy()

array([0.96650124], dtype=float32)

In [104]:
model.type_logvars(torch.tensor(types)).detach().exp().numpy().argmax()

1122

In [105]:
idx2word.get(test_idx2realidx.get(1122))

'voting'

In [157]:
# w2v.wv.index2word

In [155]:
w2v.wv.syn0[0]

/anaconda3/envs/math689env/lib/python3.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  if __name__ == '__main__':


array([ 4.7332592e-02,  5.2747063e-02, -5.1449113e-02, -4.7027846e-03,
        5.4603245e-02, -1.2933372e-01,  1.5721838e-01,  7.5663537e-02,
       -6.2025711e-02,  1.7656371e-02,  2.5907028e-01,  3.4677573e-02,
       -8.4819712e-02, -2.6605096e-01,  1.3948311e-01,  8.4106237e-02,
       -2.1419908e-04,  1.3475612e-01,  2.2660490e-02,  9.7617179e-02,
       -1.5277818e-01,  1.2325918e-02,  1.0303870e-01, -3.8719811e-02,
       -1.9681072e-01, -3.2387976e-02,  1.6129054e-02, -5.6416623e-02,
        1.3454187e-01,  2.9165758e-02], dtype=float32)

In [156]:
w2v.wv.get_vector("the")

array([ 4.7332592e-02,  5.2747063e-02, -5.1449113e-02, -4.7027846e-03,
        5.4603245e-02, -1.2933372e-01,  1.5721838e-01,  7.5663537e-02,
       -6.2025711e-02,  1.7656371e-02,  2.5907028e-01,  3.4677573e-02,
       -8.4819712e-02, -2.6605096e-01,  1.3948311e-01,  8.4106237e-02,
       -2.1419908e-04,  1.3475612e-01,  2.2660490e-02,  9.7617179e-02,
       -1.5277818e-01,  1.2325918e-02,  1.0303870e-01, -3.8719811e-02,
       -1.9681072e-01, -3.2387976e-02,  1.6129054e-02, -5.6416623e-02,
        1.3454187e-01,  2.9165758e-02], dtype=float32)

In [160]:
w2v.wv.index2word
w2v_word2idx = {w: idx for (idx , w) in enumerate(w2v.wv.index2word)}

In [115]:
flattened_tokenized_corpus = tools.flatten(tokenized_corpus)
len(np.unique(flattened_tokenized_corpus)) == w2v.syn0.shape[0]

/anaconda3/envs/math689env/lib/python3.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  from ipykernel import kernelapp as app


True

In [166]:
w2v_cos_sim_matrix = cosine_similarity(w2v.wv.syn0, w2v.wv.syn0)
def w2v_n_closest_words(word, cos_sim_matrix, n):
    word_index = w2v_word2idx.get(word)
    print("word_index is " + str(word_index))
    close_words_indices = np.argsort(cos_sim_matrix[word_index])[-n:]
#     print("close_words_indices.shape" + str(close_words_indices.shape))
    return [w2v.wv.index2word[j] for j in close_words_indices]

/anaconda3/envs/math689env/lib/python3.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  if __name__ == '__main__':


In [167]:
w2v_n_closest_words("mental", w2v_cos_sim_matrix, 20)

word_index is 87


['rather',
 'students',
 'time',
 'yet',
 'society',
 'restrained',
 'hamas',
 'president',
 'wanted',
 'peer',
 'operation',
 'animals',
 'campus',
 'set',
 'sexual',
 'providing',
 'engagement',
 'one',
 'health',
 'mental']